In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [26]:
from gensim.models import word2vec
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier

In [4]:
df = pd.read_csv('ready for classification').drop('Unnamed: 0', axis =1)
df.head()

,author,contents,description,publisher,source_url,title,date,time,label,c,marks
0,Stripe.com,a complete payments platform engineered growth...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,Ending Bitcoin Support,2018-01-23,00:00:00,0.0,"['a', 'complete', 'payments', 'platform', 'eng...",0
1,Courtney Goldsmith,tech investor tmt investments planning ramp in...,Tech investor TMT Investments is planning to r...,Cityam.com,http://www.cityam.com/282328/tech-investor-tmt...,Tech investor TMT is ready to ride the blockch...,2018-03-15,12:40:00,0.0,"['tech', 'investor', 'tmt', 'investments', 'pl...",0
2,Ricardo Esteves,bitcoin news price information analysis john m...,"John McAfee, founder of the software company M...",Newsbtc.com,https://www.newsbtc.com/2018/03/15/john-mcafee...,John McAfee Joins ‘Hackproof’ Startup CryptoSe...,2018-03-15,12:45:48,0.0,"['bitcoin', 'news', 'price', 'information', 'a...",0
3,Steve McCaskill,by steve mccaskill t z networking colt pccw gl...,Colt and PCCW Blockchain trial claims to have ...,TechRadar,https://www.techradar.com/news/blockchain-can-...,Blockchain can 'speed up' payment settelements...,2018-03-15,13:00:42,0.0,"['by', 'steve', 'mccaskill', 't', 'z', 'networ...",0
4,Lindsay Rowntree,lindsay rowntree th mar news comments blockcha...,Blockchain is a revolutionary technology that ...,Exchangewire.com,https://www.exchangewire.com/blog/2018/03/15/b...,"Blockchain Can Clean Up Programmatic, But We H...",2018-03-15,13:00:44,0.0,"['lindsay', 'rowntree', 'th', 'mar', 'news', '...",0


In [59]:
len(df[df.marks==1])

30024

In [60]:
len(df)

45217

In [6]:
#splitting the data into train and test
train, test = train_test_split(df, test_size=0.15, random_state= 67)

In [7]:
train_Y = train.marks

In [8]:
test_Y = test.marks

# Starting the Feature Selection Process

In [25]:
#performing Tf-idf on title
titles= list(train['title'])
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True)
tfidf_titles = tfidf.fit_transform(titles)


In [35]:
#for testing set
titles_test= list(test['title'])



tfidf_titles_test = tfidf.fit_transform(titles_test)

In [27]:
## modified the code from open source website http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/ 
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [28]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [29]:

model = word2vec.Word2Vec(train['c'], min_count=15)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))


In [36]:
model_test = word2vec.Word2Vec(test['c'], min_count=15)
w2v_test = dict(zip(model_test.wv.index2word, model_test.wv.syn0))

In [30]:
#tf_idf with word to vec
t = TfidfEmbeddingVectorizer(w2v)
t.fit(train['c'])
X = t.transform(train['c'])

In [37]:
#the same for test
t_test = TfidfEmbeddingVectorizer(w2v_test)
t_test.fit(test['c'])
X_test = t_test.transform(test['c'])

In [54]:
X.shape

(38434, 100)

In [53]:
tfidf_titles.shape

(38434, 13237)

In [ ]:
np.append(X_test, np.array([binary_html_test]).T,1)

In [51]:
features = np.append(X, np.array([tfidf_titles]).T)

In [52]:
features.shape

(3843401,)

In [39]:
features_test = np.append(X_test, tfidf_titles_test)

In [43]:
features.shape

(3843401,)

In [55]:
#training random forest
from sklearn.ensemble import RandomForestClassifier
rf_d = RandomForestClassifier(n_estimators=1000)
rf_d.fit(X, train_Y)



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [57]:
print('training accuracy is: ', rf_d.score(X, train_Y))

training accuracy is:  0.857574023


In [58]:
print('testing accuracy is:', rf_d.score(X_test, test_Y))

testing accuracy is: 0.34040984815
